# IDF値によるベクトライズの検討（part2）

## (1) TfidfVectorizer引数変更

プロダクションコードを以下のように修正し、nosetestsを流してみました。

- 修正前

> vectorizer = TfidfVectorizer(use_idf=False, token_pattern=u'(?u)\\b\\w+\\b')

- 修正後

> vectorizer = TfidfVectorizer(use_idf=True, token_pattern=u'(?u)\\b\\w+\\b')

## (2) 結果の比較

### (2-1) accuracyの変化

全体的に大きな変化は見られませんでした。

| テストデータ | accuracy（改修前） | <> | accuracy（改修後） |
| :--- | :---: | :---: | :---: |
| test_daikin_conversation | 0.985786336543 | > | 0.983952315452 |
| test_ptna_conversation | 0.977212971078 | == | 0.977212971078 |
| test_toyotsu_human_conversation | 0.993685872139 | == | 0.993685872139 |
| test_benefitone_conversation | 0.979611650485 | < | 0.981553398058 |

### (2-2) probabilityの変化

こちらはテストケースごとに揺れが確認されております。

特にラベル０に対するprobaの低下が目立ちます。

| テストデータ | テストケース（質問文） | 正解ラベル | proba（改修前） | <> | proba（改修後） | 
| :--- |  :--- | :---: | :---: | :---: | :---: |
| test_daikin_conversation | しいたけは嫌いな食べ物です |0| 0.9816022499009649 | > | 0.94267740147525847 |
| test_daikin_conversation| 社外からのメール添付ファイルの受信容量制限は？ | 3487 | 0.8859353165274696 | < | 0.92000754300247356 |
| test_daikin_conversation | メールに署名を付ける方法を知りたい | 3398 | 0.89561349205672769 | < | 0.94307403979364623 |
| test_daikin_conversation | SPAMフォルダにメールが入っている | 3701 | 0.98016558676347543 | > | 0.95585264818584637 |
| test_ptna_conversation | (blank) | 0 | 0.64910771108380338 | > | 0.45584103552319499 |
| test_ptna_conversation | おいしいラーメンが食べたいです | 0 | 0.64910771108380338 | > | 0.45584103552319499 |
| test_ptna_conversation | こんにちは | 2684 | 0.60717161230416883 | > | 0.60573334670493106 |
| test_ptna_conversation | 女の先生 | 4068 | 0.98318553678782494 | < | 0.98825133586199254 |
| test_ptna_conversation | 入会したいのですが | 2688 | 0.90292395657147917 | > | 0.89880333802070855 |
| test_toyotsu_human_conversation | VISAの勘定科目がわからない | 6767 | 0.25251916431211241 | < | 0.65454223963951352 |
| test_toyotsu_human_conversation | JAL マイレージ | 6803 | 0.90142687232029783 | < | 0.93270256359004111 |
| test_toyotsu_human_conversation | 海外の出張費を精算したい | 6763 | 0.58534247570230802 | < | 0.67278725326967714 |
| test_benefitone_conversation | EXカードを貸してください | 4678 | 0.98849019014720063 | < | 0.99291064405716734 |
| test_benefitone_conversation | 契約書を見たいのですが | 4683 | 0.98824130181911307 | < | 0.991716952819228 |

### (2-3) ご参考：ベクトライザーから生成されるベクトルの変化

アプリケーションログにより確認したものです。

改修前はfeatureを通じてフラットだったベクトルが、featureごとに凹凸のあるベクトルに変化しています。

### 改修前

上が学習セットの質問文（feature重複あり）、下が未知の質問文（feature重複無し）

feature（0, 1181) の要素が、学習セットと未知の質問文とでかけ離れていることが確認されます。

このため、正解ラベルに対するproba低下を誘発しやすい状況だったと推測します。

```
learning.log: DEBUG: Reply#perform text_array.separated_sentences: ['ＶＩＳＡ ＶＩＳＡ 取得 代金 勘定 科目']
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: Reply#perform features:   (0, 283)	0.353553390593
  (0, 404)	0.353553390593
  (0, 435)	0.353553390593
  (0, 865)	0.353553390593
  (0, 1181)	0.707106781187
learning.log: DEBUG: question: 「VISA」VISA取得のための代金はどの勘定科目なのか？

learning.log: DEBUG: Reply#perform text_array.separated_sentences: ['ＶＩＳＡ 勘定 科目 わかる ない']
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: Reply#perform features:   (0, 80)	0.4472135955
  (0, 122)	0.4472135955
  (0, 404)	0.4472135955
  (0, 865)	0.4472135955
  (0, 1181)	0.4472135955
learning.log: DEBUG: question: VISAの勘定科目がわからない
```

### 改修後

上が学習セットの質問文（feature重複あり）、下が未知の質問文（feature重複無し）

feature（0, 1181) の要素が、学習セットと未知の質問文とで大差がなくなったことが確認されます。

このため、正解ラベルに対するprobaが上昇したと推測します。

```
learning.log: DEBUG: Reply#perform text_array.separated_sentences: ['ＶＩＳＡ ＶＩＳＡ 取得 代金 勘定 科目']
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: Reply#perform features:   (0, 1181)	0.782861031768
  (0, 865)	0.22192839986
  (0, 435)	0.323519674117
  (0, 404)	0.223238185196
  (0, 283)	0.428224384412
learning.log: DEBUG: question: 「VISA」VISA取得のための代金はどの勘定科目なのか？

learning.log: DEBUG: Reply#perform text_array.separated_sentences: ['ＶＩＳＡ 勘定 科目 わかる ない']
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: Reply#perform features:   (0, 1181)	0.616700206566
  (0, 865)	0.349649004057
  (0, 404)	0.351712575635
  (0, 122)	0.486772196609
  (0, 80)	0.369834452061
learning.log: DEBUG: question: VISAの勘定科目がわからない
```